<a href="https://colab.research.google.com/github/ipeirotis/autoencoders_census/blob/main/Best_Trained_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf autoencoders_census
!git clone https://github.com/ipeirotis/autoencoders_census.git
%cd autoencoders_census

In [ ]:
!pip install -q import_ipynb keras-tuner
import import_ipynb
from autoencoder import AutoencoderModel
from pandas2vector import DataTransformer
from data_loader import DataLoader

In [9]:
data_loader = DataLoader()
original_df = data_loader.load_original_data()
project_data, variable_types = data_loader.prepare_original_dataset(original_df)

data_transformer = DataTransformer(variable_types)
transformed_df = data_transformer.transform_dataframe(project_data)

In [ ]:
model = AutoencoderModel()
X_train, X_test = model.split_train_test(transformed_df)

tuner = model.define_tuner()

tuner.search(X_train, X_train, epochs=model.EPOCHS,
             # batch_size=model.BATCH_SIZE,
             validation_data=(X_test, X_test))

best_hps = tuner.get_best_hyperparameters(num_trials=model.NUM_TRIALS)[0]
final_model = model.build_autoencoder(best_hps)

history = final_model.fit(X_train, X_train, epochs=5*model.EPOCHS, verbose=1, validation_data=(X_test, X_test))

model_filename = "Hyperparameter.h5"
final_model.save(model_filename)

hyperparameters_dict = {
    "learning_rate": best_hps.get('learning_rate'),
    "batch_size": best_hps.get('batch_size'),
    "num_epochs": 10,
    "loss": history.history['loss'][-1],
    "val_loss": history.history['val_loss'][-1]
}